In [ ]:
%matplotlib inline
from collections import OrderedDict
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math
import os
from scipy.optimize import brentq
from sklearn.neighbors import KernelDensity
import xarray as xr

from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import KalmanFilter as KF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.common import Q_continuous_white_noise

from pumahu import Forwardmodel, ns_sampling, LakeData, get_data
from pumahu.visualise import trellis_plot, TrellisPlot, mcmc_heat_input, heat_vs_rsam, plot_qin_uks
from pumahu.uks import UnscentedKalmanSmoother

In [ ]:
def xr2xr(xdfin):
    """
    Helper function to convert old netcdf file structure to new
    """
    new_exp = np.zeros((xdfin['exp'].shape[0], xdfin['exp'].shape[1], 2))
    new_exp[:, :, 0] = xdfin['exp'].values[:]
    new_exp[:, :, 1] = xdfin['var'].values[:]

    return xr.Dataset({'exp': (('dates', 'parameters', 'val_std'), new_exp)},
                      {'dates': xdfin.dates.values,
                       'parameters': xdfin.parameters.values,
                       'val_std': ['val', 'std']})

## Table of contents
---------------------------
1. [Inversion](#Inversion)
2. [Forecasting](#Forecasting)

## Inversion

In [ ]:
startdate = datetime.utcnow()-timedelta(days=30)
enddate = datetime.utcnow()
res_dir = './data'
if not os.path.isdir(res_dir):
    os.makedirs(res_dir)
mout_prior = get_data('./data/outflow_prior.npz')

In [ ]:
ld = LakeData()
data = ld.get_data(startdate, enddate)

In [ ]:
data_uks = ld.get_data(startdate, enddate, smoothing='dv')

In [ ]:
uks4 = UnscentedKalmanSmoother(data=data)
xds_uks = uks4()

In [ ]:
plot_qin_uks(xds_uks, annotations=True)

In [ ]:
# Setup path for results file
tstart = pd.to_datetime(data['dates'].values[0])
tend = pd.to_datetime(data['dates'].values[-1])
res_fn = 'mcmc_sampling_{:s}_{:s}.nc'
res_fn = res_fn.format(tstart.strftime('%Y-%m-%d'),
                       tend.strftime('%Y-%m-%d'))
res_fn = os.path.join(res_dir, res_fn)

ns_sampling(data, results_file=res_fn, nsamples=1001,
            nresample=500,
            q_in_lim=(0., 1000.), m_in_lim=(0., 20.),
            m_out_lim=(0., 20.), new=True,
            m_out_prior=mout_prior, tolZ=1e-3, 
            tolH=3., H=6., ws=4.5, seed=-1, intmethod='euler',
            gradient=False)

In [ ]:
xdf = xr.open_dataset(res_fn)

In [ ]:
trellis_plot(xds_uks, data2=xdf)

In [ ]:
_ = mcmc_heat_input(xdf, filename='heat.png')

In [ ]:
_ = heat_vs_rsam(xdf)

In [ ]:
#benchmark_file = './data/benchmarking/forward_2019-01-01_2021-01-11.nc'
benchmark_file = None
xdft = None
if benchmark_file is not None:
    xdft = xr2xr(xr.open_dataset(benchmark_file))

## Run the estimate for a whole year

In [ ]:
runyear = False

In [ ]:
if runyear: 
    startdate_year = datetime.utcnow()-timedelta(days=365)
    enddate_year = datetime.utcnow()
    ld = LakeData()

In [ ]:
if runyear:
    data_mcmc = ld.get_data(startdate_year, enddate_year)
    xds_mcmc = ns_sampling(data_mcmc, nsamples=1001, nresample=500,
                           q_in_lim=(0., 1000.), m_in_lim=(0., 20.),
                           m_out_lim=(0., 20.), new=False, tolZ=1e-3, 
                           tolH=3., H=6., ws=4.5, seed=-1, intmethod='euler',
                           gradient=False, results_file='heat_mcmc.nc')

In [ ]:
if runyear:
    data_uks_year = ld.get_data(startdate_year, enddate_year, smoothing='dv')
    Q = OrderedDict(T=1e-1, M=1e1, X=1e-3, q_in=1e2,
                    m_in=1e1, m_out=1e1, h=1e-3, W=1e-3,
                    dqi=0, dMi=0, dMo=0, dH=0, 
                    dW=0)
    Q = np.eye(len(Q))*list(Q.values())
    uks4 = UnscentedKalmanSmoother(data=data_uks_year, Q=Q)
    xds_uks_year = uks4()

In [ ]:
if runyear:
    fig = trellis_plot(xds_uks_year, data2=xds_mcmc)
    fig.show()